# Реализация метода CLIP + Multibead Attention

Цель: Классифицировать каждую секунду видео как "интро/титры" (1) или "основной контент" (0)
1. **Преобразуем видео в последовательность секунд**
   Видео уже порезано до 1 кадра в секунду (1 FPS)—это оптимальный компромисс по точности и скорости .

   > 1 кадр = 1 секунда времени. Длина эпизода в секундах = число кадров папки.

2. **Формируем скользящие окна по 60 кадров**
   *Окно* длиной ровно 1 мин. прокатываем по всей серии без пропусков:

   ```
   кадры 0-59, 1-60, 2-61, …  
   ```

   Такое «окно» даёт модели вид сразу на минуту истории и подходящее число параметров .

3. **Извлекаем признаки CLIP для каждого кадра**
   Каждый кадр 224×224 пропускаем через изображ-ный энкодер CLIP-ViT-B/32 и получаем вектор `f_t` размерности 512 .
   Для 60 кадров окна складывается матрица `(60 × 512)`.

4. **Добавляем обучаемые позиционные эмбеддинги**
   Чтобы модель «знала», какой кадр раньше/позже, к каждому вектору `f_t` прибавляем свой маленький вектор `P_t` (учится вместе с моделью) .

5. **Прогоняем окно через 16-слойный блок Multi-Head Attention**
   16 слоёв × 16 голов—эта глубина показала максимум F1 без переобучения .
   Attention позволяет «видеть» долгие зависимости: например, титры длятся десятки кадров.

6. **Классифицируем каждый кадр независимо**
   На выходе - 60 однонейронных «голов» (линейные слои) с сигмоидой, каждая говорит:

   ```
   ŷ_t  ≈ 1  -> кадр = интро/титры  
   ŷ_t  ≈ 0  -> кадр = основное содержание
   ```

7. **Обучаемся по бинарной кросс-энтропии**
   Потеря считается по каждому кадру окна .
   *Гиперпараметры из статьи*: Adam, `lr = 5e-5`, `batch_size = 8`, seed `42`.

8. **Получаем таймкоды**

   * **Конец интро** = последняя подряд секунда, где `p(t) > thr_start` (порог 0 .6).
   * **Начало титров** = первая секунда с `p(t) > thr_end` (порог 0 .55) ближе к концу серии.
     Те же пороги использовались авторами (видно на графике)

<img src="arcitecture.png" alt="architecture" width="1000"/>

9. **Метрики**
   *Кадровые*: Accuracy, Precision, Recall, F1.
   *Граничные*: абсолютная ошибка (сек) начала/конца.

10. **Визуализация**
    Строим график `p(t)` + горизонтальные пороги + вертикальные линии, как на вашем примере.

### Импорт и базовые настройки

In [3]:
import os
import re
import pandas as pd
import numpy as np
import torch
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import CLIPProcessor, CLIPModel
import warnings, os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # глушим низкоуровневые логи TF
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning,
                        module="transformers.image_processing_utils")
torch.manual_seed(42)
np.random.seed(42)
num = np.random.randint(0, 1000)
# Конфигурация
DATA_ROOT = Path('.')
FRAMES_DIR = DATA_ROOT / 'data' / 'frames'
AUDIO_DIR = DATA_ROOT / 'data' / 'audio'
CLIP_DIR = DATA_ROOT / 'data' / 'clip_features'  # Новая папка для CLIP-фичей
LABELS_PATH = DATA_ROOT / 'data' / 'labels.csv'
MODEL_SAVE_PATH = f"models/clip_transformer_model_{num}.pth"

# Создаем папки
CLIP_DIR.mkdir(parents=True, exist_ok=True)

# Гиперпараметры
WINDOW_SIZE = 60
BATCH_SIZE = 8
LR = 5e-5
EPOCHS = 10

# Определение устройства
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Загрузка CLIP модель
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained(
    "openai/clip-vit-base-patch32"
)
clip_model = clip_model.to(device).eval()


def time_to_seconds(time_str):
    """Конвертирует время в формате HH:MM:SS в секунды"""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)


# Загрузка разметки
labels_df = pd.read_csv(LABELS_PATH)

# Преобразуем временные метки в секунды
labels_df['start_sec'] = labels_df['start_main'].apply(time_to_seconds)
labels_df['end_sec'] = labels_df['end_main'].apply(time_to_seconds)

# Извлекаем название шоу и эпизода
labels_df['show'] = labels_df['file'].apply(lambda x: x.split('/')[0])
labels_df['episode_name'] = labels_df['file'].apply(
    lambda x: os.path.splitext(os.path.basename(x))[0]
)

Using device: cpu


### Загрузка и предобработка данных

In [4]:
# Функция для проверки полноты CLIP-фичей
def check_clip_features_complete(show, episode_name):
    """Проверяет, все ли CLIP-фичи для эпизода уже извлечены"""
    frames_dir = FRAMES_DIR / show / episode_name
    clip_dir = CLIP_DIR / show / episode_name

    if not frames_dir.exists() or not clip_dir.exists():
        return False

    frame_files = set(f.stem for f in frames_dir.glob('*.jpg'))
    clip_files = set(f.stem for f in clip_dir.glob('*.npy'))

    return frame_files == clip_files


# Функция для извлечения отсутствующих CLIP-фичей
def extract_missing_clip_features():
    """Извлекает только отсутствующие CLIP-фичи"""
    for idx, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Checking CLIP features"):
        show = row['show']
        episode_name = row['episode_name']
        frames_dir = FRAMES_DIR / show / episode_name

        if not frames_dir.exists():
            print(f"Папка с фреймами не найдена: {frames_dir}")
            continue

        # Проверяем, все ли фичи уже извлечены
        if check_clip_features_complete(show, episode_name):
            continue

        # Папка для сохранения CLIP-фичей
        clip_episode_dir = CLIP_DIR / show / episode_name
        clip_episode_dir.mkdir(parents=True, exist_ok=True)

        # Обрабатываем каждый кадр
        for frame_path in sorted(frames_dir.glob('*.jpg'), key=lambda x: int(x.stem)):
            clip_path = clip_episode_dir / f"{frame_path.stem}.npy"

            # Пропускаем уже существующие фичи
            if clip_path.exists():
                continue

            try:
                img = Image.open(frame_path).convert('RGB')
                inputs = clip_processor(images=img, return_tensors="pt").to(device)

                with torch.no_grad():
                    outputs = clip_model.get_image_features(**inputs)

                # Сохраняем фичи
                features = outputs.cpu().numpy()
                np.save(clip_path, features)

            except Exception as e:
                print(f"Ошибка обработки {frame_path}: {e}")
                # Создаем нулевой вектор в случае ошибки
                np.save(clip_path, np.zeros((1, 512)))


# Вызываем функцию для извлечения отсутствующих фичей
print("Проверка и извлечение отсутствующих CLIP-фичей...")
extract_missing_clip_features()


class ClipFeaturesDataset(torch.utils.data.Dataset):
    """Датасет для загрузки предварительно вычисленных CLIP-фичей"""

    def __init__(self, labels_df):
        self.samples = []  # (путь к фичам, метка, show, episode_name)

        for idx, row in labels_df.iterrows():
            show = row['show']
            episode_name = row['episode_name']
            start_sec = row['start_sec']
            end_sec = row['end_sec']

            clip_dir = CLIP_DIR / show / episode_name
            if not clip_dir.exists():
                print(f"Папка CLIP-фичей не найдена: {clip_dir}")
                continue

            # Получаем список фичей
            clip_files = sorted(clip_dir.glob('*.npy'), key=lambda x: int(x.stem))

            # Создаем семплы
            for i, clip_path in enumerate(clip_files):
                time_sec = i
                label = 1 if (time_sec < start_sec or time_sec > end_sec) else 0
                self.samples.append((clip_path, label, show, episode_name))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        clip_path, label, show, episode_name = self.samples[idx]
        features = np.load(clip_path)
        return torch.tensor(features).squeeze(0), label


# Создаем датасет
print("Создание датасета...")
full_dataset = ClipFeaturesDataset(labels_df)
shows = labels_df['show'].unique()

# Собираем статистику по кадрам
show_frame_counts = {}
for sample in full_dataset.samples:
    _, _, show, _ = sample
    show_frame_counts[show] = show_frame_counts.get(show, 0) + 1

print("Статистика по кадрам:")
for show, count in show_frame_counts.items():
    print(f"{show}: {count} кадров")

# Разделение эпизодов внутри каждого шоу
train_samples = []
test_samples = []

for show in shows:
    # Получаем все эпизоды этого шоу
    episodes = set()
    for sample in full_dataset.samples:
        _, _, s, ep = sample
        if s == show:
            episodes.add(ep)

    episodes = sorted(episodes)
    split_idx = int(0.8 * len(episodes))
    train_episodes = episodes[:split_idx]
    test_episodes = episodes[split_idx:]

    print(f"{show}: {len(train_episodes)} эпизодов в train, {len(test_episodes)} в test")

    # Распределяем сэмплы
    for sample in full_dataset.samples:
        _, _, s, ep = sample
        if s == show:
            if ep in train_episodes:
                train_samples.append(sample)
            elif ep in test_episodes:
                test_samples.append(sample)


# Создаем новые датасеты
class FilteredDataset(torch.utils.data.Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        clip_path, label, _, _ = self.samples[idx]
        features = np.load(clip_path)
        return torch.tensor(features).squeeze(0), label


train_data = FilteredDataset(train_samples)
test_data = FilteredDataset(test_samples)

print(f"Размер train датасета: {len(train_data)}")
print(f"Размер test датасета: {len(test_data)}")

Проверка и извлечение отсутствующих CLIP-фичей...


Checking CLIP features: 100%|██████████| 19/19 [00:00<00:00, 43.78it/s]


Создание датасета...
Статистика по кадрам:
show1: 18640 кадров
show2: 37141 кадров
show1: 4 эпизодов в train, 2 в test
show2: 10 эпизодов в train, 3 в test
Размер train датасета: 40080
Размер test датасета: 15701


### Подготовка последовательностей (скользящие окна)

In [5]:
def create_sequences(features, labels, window_size=60):
    sequences = []
    seq_labels = []

    for i in range(len(features) - window_size + 1):
        seq = features[i:i + window_size]
        lbl = labels[i:i + window_size]
        sequences.append(seq)
        seq_labels.append(lbl)

    return np.array(sequences), np.array(seq_labels)


# Собираем фичи и метки для train
train_features = []
train_labels = []
for features, label in train_data:
    train_features.append(features.numpy())
    train_labels.append(label)

train_sequences, train_seq_labels = create_sequences(train_features, train_labels, WINDOW_SIZE)

# Собираем фичи и метки для test
test_features = []
test_labels = []
for features, label in test_data:
    test_features.append(features.numpy())
    test_labels.append(label)

test_sequences, test_seq_labels = create_sequences(test_features, test_labels, WINDOW_SIZE)

# Конвертация в тензоры PyTorch
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_sequences).float(),
    torch.tensor(train_seq_labels).float()
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_sequences).float(),
    torch.tensor(test_seq_labels).float()
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

### Модель Transformer

In [6]:
import torch.nn as nn


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=60):
        super().__init__()
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_len, d_model))

    def forward(self, x):
        return x + self.pos_embedding[:, :x.size(1), :]


class VideoTransformer(nn.Module):
    def __init__(self, input_dim=512, num_layers=16, nhead=16):
        super().__init__()
        self.pos_encoder = PositionalEncoding(input_dim)

        # Трансформер (16 слоев, 16 голов)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim,
            nhead=nhead,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 60 независимых классификаторов
        self.classifiers = nn.ModuleList([
            nn.Linear(input_dim, 1) for _ in range(WINDOW_SIZE)
        ])

    def forward(self, x):
        x = self.pos_encoder(x)
        x = self.transformer(x)

        outputs = []
        for i in range(WINDOW_SIZE):
            outputs.append(self.classifiers[i](x[:, i]))

        return torch.stack(outputs, dim=1).squeeze(-1)


model = VideoTransformer().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.BCEWithLogitsLoss()

### Обучение модели

In [7]:
def train_epoch(model, loader):
    model.train()
    total_loss = 0

    for batch in tqdm(loader, desc="Training"):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        # Temporal Shifting (регуляризация из статьи)
        if np.random.rand() > 0.5:
            shift = np.random.randint(-5, 6)
            inputs = torch.roll(inputs, shifts=shift, dims=1)
            labels = torch.roll(labels, shifts=shift, dims=1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


for epoch in range(EPOCHS):
    loss = train_epoch(model, train_loader)
    print(f"Epoch {epoch + 1}/{EPOCHS} | Loss: {loss:.4f}")

# Сохранение модели
torch.save(model.state_dict(), MODEL_SAVE_PATH)

Training: 100%|██████████| 5003/5003 [2:40:00<00:00,  1.92s/it]  


Epoch 1/10 | Loss: 0.1306


Training: 100%|██████████| 5003/5003 [2:27:50<00:00,  1.77s/it]  


Epoch 2/10 | Loss: 0.1400


Training: 100%|██████████| 5003/5003 [2:18:50<00:00,  1.67s/it]  


...

Training: 100%|██████████| 5003/5003 [2:37:01<00:00,  1.88s/it]  


Epoch 6/10 | Loss: 0.1399


Training: 100%|██████████| 5003/5003 [2:34:42<00:00,  1.86s/it]  


Epoch 7/10 | Loss: 0.1397


Training:  20%|██        | 1007/5003 [31:38<2:11:55,  1.98s/it]

### Оценка и метрики

In [9]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Evaluating"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
            all_preds.append(preds)
            all_labels.append(labels.numpy())

    preds = np.vstack(all_preds).flatten()
    labels = np.vstack(all_labels).flatten()

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0
    )
    acc = accuracy_score(labels, preds)

    return acc, precision, recall, f1


acc, precision, recall, f1 = evaluate(model, test_loader)
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [12]:
print(123)

### Визуализация предсказаний 

In [10]:
def plot_predictions(episode_features, true_labels):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(episode_features).unsqueeze(0).to(device)
        outputs = model(inputs)
        probs = torch.sigmoid(outputs).cpu().numpy().flatten()

    # Постобработка: медианный фильтр
    probs_smoothed = np.convolve(probs, np.ones(5) / 5, mode='same')

    plt.figure(figsize=(15, 6))
    plt.plot(probs_smoothed, 'b-', label='Predicted Probability')
    plt.axhline(y=0.5, color='r', linestyle='--', label='Decision Threshold')
    plt.plot(true_labels, 'g-', alpha=0.5, label='True Labels')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Probability')
    plt.title('Intro/Credits Detection')
    plt.legend()
    plt.grid(True)
    plt.savefig('detection_plot.png')
    plt.show()


# Пример визуализации для первого тестового эпизода
plot_predictions(
    test_sequences[0],
    test_seq_labels[0]
)

### Определение таймкодов

In [11]:
def detect_timestamps(probs, threshold=0.5):
    """Определяет таймкоды начала и конца основного контента"""
    # Начало интро: первый кадр с prob > threshold
    start_intro = np.argmax(probs > threshold)

    # Конец интро: последний кадр перед основным контентом
    end_intro = start_intro
    while end_intro < len(probs) and probs[end_intro] > threshold:
        end_intro += 1

    # Начало титров: первый кадр титров в конце
    start_credits = len(probs) - np.argmax(probs[::-1] > threshold) - 1

    return {
        'start_main': end_intro + 1,  # начало контента после интро
        'end_main': start_credits  # конец контента перед титрами
    }


# Пример использования для первого тестового эпизода
with torch.no_grad():
    inputs = torch.tensor(test_sequences[0]).unsqueeze(0).to(device)
    outputs = model(inputs)
    probs = torch.sigmoid(outputs).cpu().numpy().flatten()

timestamps = detect_timestamps(probs)
print(f"Start Main: {timestamps['start_main']} sec")
print(f"End Main: {timestamps['end_main']} sec")